# 第 2 章 找出两篇文章的不同
*****

开发一个大模型工具，实现文档内容比对。阿里云的文档工程师们遇到过这样的问题：同一篇文档在中国站和国际站有一些不同，能快速的指出一批文档的这种差异吗？收到这个问题的你起初感到有些犯难，但想到或许可以借助大模型来快速完成这个任务，你又觉得似乎可以很快做到。

本章我们将开发一个程序，该程序可以完成：
- 可以指定文档 URL，程序根据 URL 自动爬取中国站、国际站的文档内容作比对
- 因为要比对的文档很多，你希望是可以从一个 excel 文件中读取 URL 列表，并对其进行比对


## 1. 准备工作

### 1.1. 安装

下载文档代码及安装依赖项
```bash
git clone https://github.com/AlibabaCloudDocs/llm_learning.git
cd llm_learning
pip install -r requirements.txt
```

### 1.2. 账号准备

首先，您需要前往 [官网创建 API Key](https://help.aliyun.com/zh/dashscope/developer-reference/activate-dashscope-and-create-an-api-key)。接下来，请获取你的 [DASHSCOPE_API_KEY](https://dashscope.console.aliyun.com/apiKey)

####  MacOS or Linux
您可以使用以下命令行导入环境变量
```bash
export DASHSCOPE_API_KEY="sk-****"
```

#### Windows
可以在终端使用[`SET`](https://learn.microsoft.com/zh-cn/windows-server/administration/windows-commands/set_1)命令设置环境变量
```bat
set DASHSCOPE_API_KEY=sk-****
```
或者在[`PowerShell`](https://learn.microsoft.com/zh-cn/powershell/module/microsoft.powershell.core/about/about_environment_variables?view=powershell-7.4)中使用以下命令行配置环境变量 
```powershell
$Env:DASHSCOPE_API_KEY = "sk-****"
```

#### Jupyter Notebook
您可以使用[`os.environ`](https://docs.python.org/3/library/os.html)方法，在代码开头设置临时环境变量。


## 2. MVP 开发
在正式开发一个完整的程序之前，我们可以先开发一个最小可运行的程序，来验证一下效果。

### 2.1 根据 URL 加载文档
为了实现根据 URL 读取文档内容，你参考了 LangChain 的[这篇 WebBaseLoader 文档](https://python.langchain.com/docs/integrations/document_loaders/web_base)，并尝试在代码中使用 WebBaseLoader 来加载一篇文档：

In [45]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://help.aliyun.com/zh/ecs/user-guide/create-a-subscription-instance-on-the-quick-launch-tab")

doc = loader.load()

### 2.2 比较文档内容
接下来你可以再加载一下国际站的文档，并将其交给通义千问帮你比对差别了：

In [36]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.llms import Tongyi
from utils.ds_util import sample_call_streaming

llm = Tongyi()
llm.model_name = 'qwen-max'


def load_doc_content(url):
    loader = WebBaseLoader(url)
    doc = loader.load()
    return doc[0].page_content


doc_content_cn = load_doc_content("https://help.aliyun.com/zh/ecs/user-guide/create-a-subscription-instance-on-the-quick-launch-tab")
doc_content_intl = load_doc_content("https://www.alibabacloud.com/help/zh/ecs/user-guide/create-a-subscription-instance-on-the-quick-launch-tab")

prompt = f"""你现在的任务是找出下面两篇文档的不同之处，请不要放过任何细节。
--------
【中国站的文档】：
{doc_content_cn}
--------
【国际站的文档】:
{doc_content_intl}
--------
请给出结论，并用列表形式指出不同之处的具体位置：
"""

sample_call_streaming(prompt)


以下是两篇文档的不同之处列表：

1. 文档标题：
   - 中国站文档：快速购买ECS实例_云服务器 ECS(ECS)-阿里云帮助中心
   - 国际站文档：快速购买ECS实例 - 云服务器 ECS - 阿里云

2. 页面布局与导航链接：
   - 中国站文档：文档结尾包含了联系信息、备案控制台登录/注册、产品文档、输入文档关键字查找等；还有“首页”、“云服务器 ECS”的子菜单结构。
   - 国际站文档：文档开头有“文档中心”和“全部产品”的链接，且页面结构更加简洁，无明显的子菜单结构。

3. 内容差异：
   - 创建实例过程中的默认配置：
     - 可用区分配：中国站文档提到“系统随机分配，不能修改”，而国际站文档未提及不能修改。
     - 实例规格：中国站文档提供多个实例规格选项，包括 ecs.s6-c1m1.small 等，而国际站文档仅支持突发性能实例 t5，并提及 CentOS、Windows Server、Ubuntu 三种操作系统可供选择，没有提及 Alibaba Cloud Linux。
     - 网络类型：中国站文档默认为专有网络，国际站文档同样默认为专有网络，但提到了“专有网络公网带宽”是需要额外配置的选项。
     - 公网 IP 分配：两个文档都提及必须分配公网 IP，但在计费模式描述上有差异，中国站文档有“按固定带宽”和“按使用流量”两种选择，而国际站文档只有“按固定带宽”和对应的带宽值选择。

4. 后续步骤：
   - 中国站文档提到创建实例成功后，用户需在 ECS 管理控制台重置密码，并提供了相关操作步骤链接；国际站文档同样提到重置密码，但没有给出具体的链接或操作步骤。

5. 法律声明和页面底部信息：
   - 中国站文档包含多个联系方式、备案信息、法律声明及隐私权政策、Cookies 政策、廉政举报、安全举报等内容；而国际站文档只提到感谢用户的反馈，并无详细法律声明和其他服务信息。

以上就是一个 MVP 版本的程序了。它可以根据 URL 加载文档并进行比较，一定程度上，它已经能帮你减轻很多工作量了。

## 3. 完善程序
接下来我们可以完善一下程序，以达成我们最初设想的实现目标。

### 3.1 遍历 Excel 中的文档 URL
因为要比对的文档比较多，而且也经常会变，所以在代码中写死这些链接并不是一个好的做法。 我们可以在 excel 文件中管理 url，然后在程序中读取出来。

参考代码如下：

In [37]:
import os
from openpyxl import load_workbook

file_path = 'data/doc_urls.xlsx'

wb = load_workbook(file_path)
sheet = wb.active
for row in sheet.iter_rows(values_only=True, min_row=2):
    url_cn = row[0]
    url_intl = row[0].replace('help.aliyun.com', 'www.alibabacloud.com/help')
    print(url_cn)
    print(url_intl)

https://help.aliyun.com/zh/ecs/user-guide/create-a-subscription-instance-on-the-quick-launch-tab
https://www.alibabacloud.com/help/zh/ecs/user-guide/create-a-subscription-instance-on-the-quick-launch-tab
https://help.aliyun.com/zh/ecs/user-guide/overview-52
https://www.alibabacloud.com/help/zh/ecs/user-guide/overview-52


你已经可以从 excel 中读取链接，并自动替换获取国际站的文档链接了。 我们可以对 MVP 程序进行一点封装，以便在遍历 excel 里的 url 时调用通义千问进行文档对比。 详细的代码可以查看：

详细的代码可以查看：
- [chapter2/load-from-excel/diff_docs.py](load-from-excel/diff_docs.py)
- [chapter2/load-from-excel/main.py](load-from-excel/main.py)

现在，你就可以运行程序，来实现批量自动文档对比了：


In [38]:
! python load-from-excel/main.py

------
中国站文档：https://help.aliyun.com/zh/ecs/user-guide/create-a-subscription-instance-on-the-quick-launch-tab
国际站文档：https://www.alibabacloud.com/help/zh/ecs/user-guide/create-a-subscription-instance-on-the-quick-launch-tab
结论：两篇文档分别是中国站和国际站关于快速购买包年包月ECS实例的指南，尽管主要内容相似，但存在一些细节上的不同之处。

不同之处具体位置及内容如下：

1. 文档标题：
   - 中国站文档标题：“快速购买ECS实例_云服务器 ECS(ECS)-阿里云帮助中心”
   - 国际站文档标题：“快速购买ECS实例 - 云服务器 ECS - 阿里云”

2. 更新时间：
   - 中国站文档更新时间为未明确给出
   - 国际站文档更新时间为“Nov 30, 2023”

3. 前提条件中的账号注册流程说明：
   - 中国站文档提到“已注册账号并完成实名认证。具体操作，请参见阿里云账号注册流程。”
   - 国际站文档提到“已注册账号并完成实名认证。具体操作，请参见注册阿里云账号。”

4. 实例规格部分：
   - 中国站文档列举的默认实例规格是“ecs.s6-c1m1.small”
   - 国际站文档提到一键购买仅支持“突发性能实例t5”，并且举例为“2 vCPU 4 GiB（突发性能实例t5）”

5. 操作系统支持列表：
   - 中国站文档提到支持的操作系统包括Alibaba Cloud Linux、Windows Server、Ubuntu、CentOS四种
   - 国际站文档提到一键购买支持的操作系统为CentOS、Windows Server、Ubuntu三种

6. 确认订单页面的条款说明：
   - 中国站文档提及阅读《云服务器 ECS 服务条款》|《云服务器ECS退订说明》
   - 国际站文档提及阅读《云服务器 ECS 服务条款》|《通用服务条款》

7. 文档末尾的联系方式和服务信息：
   - 中国站文档提供了详细的联系方式、权益中心、关注方式等信息
   - 国际站文档在对比范围内没有提供这些详细

### 3.2 排除网页中的页头页尾等非关键内容
细心的你可能会发现运行前面的代码时，通义千问给出的文档差异，可能还会包含一些页面尾部的差异，比如：
```
...
2. 联系方式和链接格式：
   - 中国站文档中提供了详细的联系方式，如“95187-1 在线服务”、“4008013260 在线服务”，并且附带了“我要建议”、“我要投诉”、“体验反馈”等功能入口，同时链接地址采用了中文描述且未直接展示超链接形式。
   - 国际站文档没有在该处明确提供电话联系方式，而是集中在文档底部显示版权信息时提及，并且所有链接均采用英文描述及超链接形式展现。

3. 页面底部信息：
   - 中国站文档在底部包含了更多的附加信息，如“阿里云首页”、“相关技术圈”、“为什么选择阿里云”等，并列出了各类服务热线、法律声明、隐私权政策、廉正举报等链接。
   - 国际站文档在底部只包含了版权信息、许可证编号以及指向其他阿里云产品的链接，联系方式较为简洁。
...
```

出现这个情况的原因是，我们给到大模型的文档内容，其实是整个页面的内容。为了解决这一问题，我们需要想办法提取到文档正文本身，排除页头页尾等非文档正文内容。

这里我们可以选择针对阿里云文档页面的结构，编写一个自定义的 loader：

In [39]:
from typing import List
from bs4 import BeautifulSoup
from langchain_community.document_loaders.base import BaseLoader
from langchain_community.document_transformers import Html2TextTransformer
from langchain_core.documents import Document
import requests


class AliyunDocLoader(BaseLoader):
    """
    阿里云文档加载器
    """

    def __init__(self, urls: List[str]):
        self.urls = urls

    def load(self) -> List[Document]:
        results = []
        for url in self.urls:
            content, metadata = self._get_aliyun_help_doc(url)
            doc = Document(page_content=content, metadata=metadata)
            results.append(doc)
        html2text = Html2TextTransformer()
        return html2text.transform_documents(results)

    @staticmethod
    def _get_aliyun_help_doc(url: str):
        """
        @param url: 文档 URL
        @return: content, metadata. content 为 html，metadata 里包含 TDK
        """
        html = requests.get(url).text
        soup = BeautifulSoup(html, features='html.parser')
        markdown_body = soup.find('article', class_='markdown-body') or soup.find('div', class_='markdown-body')
        contents = markdown_body.contents
        content_html = ''.join([str(item) for item in contents])
        content_html = f'<h1>{soup.find("h1").text}</h1>' + content_html

        metadata = {
            "title": soup.find('title').text,
            "keywords": soup.find('meta', attrs={'name': 'keywords'}).attrs['content'],
            "description": soup.find('meta', attrs={'name': 'description'}).attrs['content']
        }
        return content_html, metadata

完整的代码在 [`chapter2/custom-loader/`](custom-loader/) 中。
现在你可以尝试运行一下使用自定义阿里云文档 loader 的对比程序了：

In [40]:
! python custom-loader/main.py

------
中国站文档：https://help.aliyun.com/zh/ecs/user-guide/create-a-subscription-instance-on-the-quick-launch-tab
国际站文档：https://www.alibabacloud.com/help/zh/ecs/user-guide/create-a-subscription-instance-on-the-quick-launch-tab
根据对比分析，以下是两篇文档中的不同之处：

1. 文档标题：
   - 中国站文档：快速购买包年包月实例
   - 国际站文档：云服务器 ECS：一键购买包年包月实例

2. 操作步骤中关于产品规格的说明和示例：
   - 中国站文档：
     > 产品规格
     > 可选的实例规格和地域等因素有关，您可以在 **使用说明** 列，查看该实例规格适用的场景。如需更多实例规格，请前往自定义购买。
     > 示例：ecs.s6-c1m1.small
   - 国际站文档：
     > 实例规格
     > 一键购买仅支持突发性能实例t5，如需更多实例规格，请前往自定义购买。
     > 示例：2 vCPU 4 GiB（突发性能实例t5）

3. 操作系统和预装应用的选择：
   - 中国站文档提供了操作系统和预装应用两种选择，并指出二者只能选其一。
   - 国际站文档未提及预装应用选项，只提供镜像选择，且列举了三种操作系统：CentOS、Windows Server、Ubuntu。

4. 公网带宽计费模式及配置的展示方式：
   - 中国站文档将公网IP和带宽计费模式分开描述，并列举了两种带宽计费模式的具体内容。
   - 国际站文档在“公网带宽”部分同时包含了公网IP分配和带宽计费模式的选择，且整合在一个列表项中展示。

5. 购买时长和自动续费的展示方式：
   - 中国站文档将购买时长和自动续费作为两个独立参数分别列出。
   - 国际站文档将购买时长和是否启用自动续费合并在一起作为一个整体进行展示。

6. 确认订单环节中的服务条款：
   - 中国站文档提到的服务条款为《云服务器 ECS 服务条款》和《云服务器ECS退订说明》。
   - 国际站文档提到的服务条款为《云服务器 ECS 服务条款》和《通

### 3.3 应对超大文档
以上的程序似乎已经比较完善了。但很快你就会发现，有一些文档非常大，超出通义千问 API 的最大 token 数限制了。

### 3.3.1 更换模型
经过查阅阿里云的 [DashScope - 通义千问模型](https://help.aliyun.com/zh/dashscope/developer-reference/api-details)说明文档，你会发现正在使用的`qwen-max`最大 token 数是 6k。
另外，你还发现`qwen-plus`和`qwen-max-longcontext`支持的 token 数分别是 30k 和 28k。
所以最简单的办法就是修改现有的代码，使用`qwen-plus`和`qwen-max-longcontext`即可。

In [41]:
llm = Tongyi()
llm.model_name = 'qwen-max-longcontext'

尽管上面的代码改动已经解决了你遇到的问题，但严谨的你一定会想到，如果遇到了更大的文档，还是超过了 token 数量限制该怎么办。

### 3.3.2 分而治之
我们可以借鉴 MapReduce 的思想，将文档分成多个小文档分片，然后对每个分片做对比，最后汇总结果。

具体实现思路是：
1. 我们可以先找出两篇文档中相同的二级标题；
2. 然后以这些相同的二级标题对文档进行切片；
3. 对每个切片进行对比，并将结果汇总，得到最终结果。

我们在[`chapter2/long-docs/doc_spliter.py`](long-docs/doc_spliter.py)中提供了两个方法，用于找到相同的二级标题，和对文档进行切片。
另外在[`chapter2/long-docs/diff_docs.py`](long-docs/diff_docs.py)中调整了对比程序的实现，程序会先对第一个文档切片进行对比，然后带着对比结论，对剩余文档切片进行对比，最终遍历完所有的切片时，就得到了最终答案。

这是切片提问模式下的两段提示词：
```python
def get_question_prompt(doc_content_cn, doc_content_intl):
    return f"""你现在的任务是找出下面两段文档的不同之处，请不要放过任何细节。
--------
【中国站的文档】：
{doc_content_cn}
--------
【国际站的文档】:
{doc_content_intl}
--------
请给出结论，并用列表形式指出不同之处的具体位置：
"""


def get_refine_prompt(doc_content_cn, doc_content_intl, existing_answer):
    return f"""这是你在前面的文档片段中找出的不同：
{existing_answer}
--------
请找出下面两个文档片段不同之处的具体位置，并补充到之前的结果中：
--------
【中国站的文档】：
{doc_content_cn}
--------
【国际站的文档】:
{doc_content_intl}
"""
```

你可以运行一下试试看：

In [42]:
! python long-docs/main.py

------
中国站文档：https://help.aliyun.com/zh/ecs/user-guide/create-a-subscription-instance-on-the-quick-launch-tab
国际站文档：https://www.alibabacloud.com/help/zh/ecs/user-guide/create-a-subscription-instance-on-the-quick-launch-tab
正在对比第1个片段...
正在对比第2个片段...
正在对比第3个片段...
正在对比第4个片段...
经过比对，两个文档片段在内容上存在以下不同之处：

1. 创建ECS实例时选择产品规格的描述：
   - 中国站文档中：可选产品规格ecs.s6-c1m1.small，并提供了查看使用说明的提示。
   - 国际站文档中：一键购买仅支持突发性能实例t5。

2. 操作系统和预装应用部分：
   - 中国站文档中：详细列出了操作系统（包括Alibaba Cloud Linux、Windows Server、Ubuntu、CentOS）和预装应用，并特别说明二者只能选择其中一个。
   - 国际站文档中：只提及一键购买支持的操作系统镜像类型（CentOS、Windows Server、Ubuntu），未提及预装应用的选择。

3. 带宽计费模式与带宽值的呈现方式：
   - 中国站文档中：将带宽计费模式（按固定带宽或按使用流量）和带宽值分开表述。
   - 国际站文档中：公网带宽部分整合了公网IP分配、带宽计费模式选择（按固定带宽）以及具体的带宽值（1 Mbps）。

4. 购买时长和自动续费选项的顺序和表述：
   - 中国站文档中：先列出购买时长，再列出自动续费选项。
   - 国际站文档中：购买数量后面直接是购买时长及其与自动续费的一体化选择。

5. 确认订单步骤中的页面名称和协议内容：
   - 中国站文档中：在确认订单面板阅读《云服务器 ECS 服务条款》|《云服务器ECS退订说明》，然后确认订单。
   - 国际站文档中：在 **确认订单** 页面阅读《云服务器 ECS 服务条款》|《通用服务条款》，然后确认下单。
------
中国站文档：https://help.aliyun.com/zh/ecs/user-gu

## 4. 总结
通过本章的学习，你已经能借助通义千问的 API 来帮助自己完成一些原来要人工完成的工作了，也学会了通过分而治之的思路来解决提示词过长的问题。分而治之的思想在很多场景下都有应用，除了用于做两篇长文档的对比，也可以用于超长文档的总结。

需要注意的时，这里出于学习的目的，文档的切片方法并不是最优的：
- 提示词长度没有超限时，是可以不用切片的；
- 一些特殊的文档也可能存在一个二级标题下内容也超过 token 限制，实际用于生产时，需要结合具体业务来定制切片方法。

另外，在实际使用中，你也可以使用 LangChain 提供的 refine 类型的 load_summarize_chain 来完成这个切片后调用大模型的任务。详情可以参考：[LangChain Use cases：Summarization - Refine](https://python.langchain.com/docs/use_cases/summarization#option-3.-refine)。


## 5. 参考资料
- [DashScope](https://dashscope.aliyun.com/)
- [通义千问 - 模型概览](https://help.aliyun.com/zh/dashscope/developer-reference/api-details)
- [LangChain](https://python.langchain.com/docs)
- [LangChain Use cases：Summarization - Refine](https://python.langchain.com/docs/use_cases/summarization#option-3.-refine)